In [1]:
import backtrader as bt
import datetime
import yfinance as yf
from stock_config import *  
%matplotlib inline

In [2]:
cerebro = bt.Cerebro()

In [3]:
class TestStrategy(bt.Strategy):

    # 照抄就OK
    def log(self, txt, dt=None):
        ''' Logging function for this strategy
            這個功能是用作Log唔同既信息'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    # 呢個要改 - 只係行一次
    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        # 呢度係一開始要初始化既參數, 就呢個策略而言, 成個策略只需要拎到收巿價就OK,
        # 所以呢度只需要拎close
        self.dataclose = self.datas[0].close

        # To keep track of pending orders and buy price/commission
        # 呢度之後會變, 但由於會用到呢D參數, 所以一開始都要初始化左先
        # 如果你既策略有唔同既參數要用, 記得要加返係度
        self.order = None
        self.buyprice = None
        self.buycomm = None

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            # 如果個broker接受左你個order, 交易左啦, 咁呢度就乜都唔使做
            # 如果未交易, 就會check下下邊係邊個status, report返比我地知
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        # 如果你買既時候你個戶口唔夠錢, 就唔會complete, 咁呢度都唔行
        if order.status in [order.Completed]:
            # 如果你買貨
            if order.isbuy():
                self.log(
                    #'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    '已執行買入訂單, 價格: %.2f, 成本: %.2f, 佣金 %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm

            # 如果你;賣貨
            else:  # Sell
                self.log('已執行賣出訂單, 價格: %.2f, 成本: %.2f, 佣金 %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            # 呢個係用黎計算長度, 一支K線為之一, 係呢個策略度呢個唔好郁佢
            # 佢用黎計賣出時間, 下邊有講, 另外用黎計最少拎住幾耐
            self.bar_executed = len(self)

        # 如果訂單出現問題
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('訂單 取消/保証金不足/被拒')
                # 該訂單的執行將意味著追加保証金的需要, 而且, 之前所接受的訂單已從系統中刪除

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('平倉利潤, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    # 每一支K線開始就會行一次呢行野
    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('K線收巿價, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        # 如果本身有order係落緊未完成交易, 就唔買
        if self.order:
            return

        # Check if we are in the market
        # 如果我地本身無野買左係手
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            # 呢個策略係連升兩日, 今日高過琴日, 琴日又高過前日
            if self.dataclose[0] < self.dataclose[-1]:
                    # current close less than previous close

                    if self.dataclose[-1] < self.dataclose[-2]:
                        # previous close less than the previous close

                        # BUY, BUY, BUY!!! (with default parameters)
                        self.log('發出買入訂單, %.2f' % self.dataclose[0])

                        # Keep track of the created order to avoid a 2nd order
                        self.order = self.buy()

        else: # 如果有野係手

            # Already in the market ... we might sell
            # 如果拎住左5枝K就賣
            if len(self) >= (self.bar_executed + 5):
        
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('發出賣出訂單, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

In [4]:
df = yf.download('MSFT', start = '2010-01-01')
df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-01-04,30.620001,31.100000,30.590000,30.950001,23.800220,38409100
2010-01-05,30.850000,31.100000,30.639999,30.959999,23.807913,49749600
2010-01-06,30.879999,31.080000,30.520000,30.770000,23.661810,58182400
2010-01-07,30.629999,30.700001,30.190001,30.450001,23.415728,50559700
2010-01-08,30.280001,30.879999,30.240000,30.660000,23.577217,51197400
...,...,...,...,...,...,...
2022-06-27,268.209991,268.299988,263.279999,264.890015,264.890015,24615100
2022-06-28,263.980011,266.910004,256.320007,256.480011,256.480011,27295500
2022-06-29,257.589996,261.970001,255.759995,260.260010,260.260010,20069800


In [5]:
feed = bt.feeds.PandasData(dataname = df) #記得要加s, PandasData唔係Pandas, dataname 唔係 dataframe
cerebro.adddata(feed)

In [6]:
cerebro.addstrategy(TestStrategy)

0

In [7]:
cerebro.run()

2010-01-04, K線收巿價, 30.95
2010-01-05, K線收巿價, 30.96
2010-01-06, K線收巿價, 30.77
2010-01-07, K線收巿價, 30.45
2010-01-07, 發出買入訂單, 30.45
2010-01-08, 已執行買入訂單, 價格: 30.28, 成本: 30.28, 佣金 0.00
2010-01-08, K線收巿價, 30.66
2010-01-11, K線收巿價, 30.27
2010-01-12, K線收巿價, 30.07
2010-01-13, K線收巿價, 30.35
2010-01-14, K線收巿價, 30.96
2010-01-15, K線收巿價, 30.86
2010-01-15, 發出賣出訂單, 30.86
2010-01-19, 已執行賣出訂單, 價格: 30.75, 成本: 30.28, 佣金 0.00
2010-01-19, 平倉利潤, GROSS 0.47, NET 0.47
2010-01-19, K線收巿價, 31.10
2010-01-20, K線收巿價, 30.59
2010-01-21, K線收巿價, 30.01
2010-01-21, 發出買入訂單, 30.01
2010-01-22, 已執行買入訂單, 價格: 30.00, 成本: 30.00, 佣金 0.00
2010-01-22, K線收巿價, 28.96
2010-01-25, K線收巿價, 29.32
2010-01-26, K線收巿價, 29.50
2010-01-27, K線收巿價, 29.67
2010-01-28, K線收巿價, 29.16
2010-01-29, K線收巿價, 28.18
2010-01-29, 發出賣出訂單, 28.18
2010-02-01, 已執行賣出訂單, 價格: 28.39, 成本: 30.00, 佣金 0.00
2010-02-01, 平倉利潤, GROSS -1.61, NET -1.61
2010-02-01, K線收巿價, 28.41
2010-02-02, K線收巿價, 28.46
2010-02-03, K線收巿價, 28.63
2010-02-04, K線收巿價, 27.84
2010-02-05, K線收巿價, 28.02
2010-02-08, 